In [6]:
import os
import pickle
import numpy as np
import pandas as pd
import cv2
from collections import defaultdict
from tqdm import tqdm
from fontTools.ttLib import TTFont
from PIL import Image, ImageDraw, ImageFont

In [7]:
def get_image(font, char, new_size=128):
    img = Image.new('L', (1000,1000), 255)

    draw = ImageDraw.Draw(img)
    draw.text((200,200), char, font=font)

    npimg = 255 - np.array(img)
    wmin = npimg.sum(0).nonzero()[0].min()
    wmax = npimg.sum(0).nonzero()[0].max()
    hmin = npimg.sum(1).nonzero()[0].min()
    hmax = npimg.sum(1).nonzero()[0].max()

    npimg = 255 - npimg[hmin:hmax+1,wmin:wmax+1]
    img = Image.fromarray(npimg)
    
    padded_img = Image.new("L", (new_size, new_size), 255)
    left = (padded_img.width - img.width) // 2
    top = (padded_img.height - img.height) // 2
    right = left + img.width
    bottom = top + img.height
    
    padded_img.paste(img, (left, top, right, bottom))
    
    return padded_img

def get_existing_chars(filename):
    ttfont = TTFont(filename)
    return list({chr(key) for table in ttfont['cmap'].tables for key in table.cmap.keys()})


In [8]:
ak = '''가 깨 냐 댸 떠 레 며 볘 삐 사 쌔 야 쟤 쩌 체 켜 톄 피 하 교 꾸 뉴 드 또 료 무 뷰 쁘 소 쑈 우 쥬 쯔 초 쿄 투 
퓨 흐 귀 끠 놔 돼 뙤 뤄 뭬 뷔 쁴 솨 쐐 외 줘 쮀 취 킈 톼 퐤 회 걕 꺾 넧 뎐 뗹 릲 몯 뵬 뿕 슒 쓻 왌 좵 쬞 춿 퀨 튑 픲 핫 갰 꺙 넂 덫 뗔 렽 몦 빟'''
ak = ak.replace("\n","").split(" ")

In [9]:
ttffd = "/home/jupyter/ai_font/data/train_ttfs"
fd = "/home/jupyter/ai_font/data/exp0717/train_whole/"

os.makedirs(fd, exist_ok=True)
fontfiles = sorted([(f.replace(".ttf",""), f"{ttffd}/{f}") for f in os.listdir(ttffd) if f.endswith(".ttf")])

In [10]:
font_size = 64
pbar = tqdm([f for i,f in enumerate(fontfiles)])
for fontname, filename in pbar:

    font = ImageFont.truetype(filename,font_size)
    existing_chars = get_existing_chars(filename)
    iter_chars = list(set(existing_chars).intersection(set(ak)))
    
    os.makedirs(f"{fd}/{fontname}", exist_ok=True)
    savecount = 0
    for k in iter_chars:
        try:
            img = get_image(font, k)
            img.save(f"{fd}/{fontname}/{fontname}__{k}.png")
            pbar.set_postfix(n_files=f"{savecount}/{len(iter_chars)}", font=fontname)
            savecount += 1
        except:
            pass

100%|██████████| 426/426 [02:37<00:00,  2.70it/s, font=프리텐다드중간체, n_files=83/84]            


In [6]:
fontname

'프리텐다드중간체'

In [8]:
img.size

(36, 32)